In [1]:
import numpy  as np
import pandas as pd
import fnmatch
import os
import re

import numpy  as np
import matplotlib.pyplot as plt
from os.path import splitext as ossplitext


In [2]:
def find_files(path_main, pattern, type='separate'):

    list_return = list()
    for dirpath, dirs, files in os.walk(path_main):
        for fname in fnmatch.filter(files, pattern):
            list_return.append((dirpath,fname))

    if type == 'separate':
        list_return = np.asarray(list_return)
        df 			= pd.DataFrame(list_return, columns = ['path','file'])
        df 			= df.sort_values(by=['path'], ascending=True)
        return df.to_numpy()

    if type == 'absolute':
        new_list = list()
        for i,j in list_return:
            new_list.append(os.path.join(i,j))
        new_list = sorted(new_list )
        return np.asarray(new_list)
    else:
        print('error, you need choise type: [separate,absolute]')
        return None

In [3]:

def build_dataframe(files):
    cols = ['queryAbsPath','qId','qIsPesentGT','beta','eta','tau','highSimilarity','qIsIn_eta','type']
    main_df = pd.DataFrame(columns=cols)

    for i in files:
        df = pd.read_csv(i, usecols=cols)
        if df.shape[0]>0:
            main_df = main_df.append(df, ignore_index=True)
    
    return main_df

In [4]:

def build_outcome_dataframe(files):
#     cols = ['imgAbsPath','cosine_similarity']
    cols = ['imgAbsPath_origin','imgAbsPath_current','cosine_similarity']
    main_df = pd.DataFrame(columns=cols)

    for i in files:
        df = pd.read_csv(i, usecols=cols)
        if df.shape[0]>0:
            main_df = main_df.append(df, ignore_index=True)
    return main_df

In [5]:
def create_dir(path):
    try:
        os.makedirs(path, exist_ok=True)
        # print('{} created successfully'.format(path))
        return path
    except:
        print('{} cant be created')
    return None

In [6]:
xcols = ['imgAbsPath_origin','imgAbsPath_current','cosine_similarity']
xmain_df = pd.DataFrame(columns=xcols)
xmain_df.shape

(0, 3)

# PATHs

In [19]:
path_main   = './../dataset_prid2011/1_yolov3_reid_every_80_frames'
path_save   = './results/1_yolov3_reid_every_80_frames'

In [20]:
create_dir(path_save)

'./results/2_trade_reid'

In [9]:
def gather_outcome_dataframes(path_main, path_save, prefix_reid, thisCam=False, otherCam=True):
    if thisCam:
        outcomefilesThisCam  = find_files(path_main, '{}_outcome_thisCam_person_*_dataframe.csv'.format(prefix_reid), type='absolute')
        df_outcome_thisCam   = build_outcome_dataframe(outcomefilesThisCam)
        print("df_outcome_thisCam shape: ",df_outcome_thisCam.shape)
        df_outcome_thisCam.to_csv(os.path.join(path_save,'skip_gather_df_outcomes_thisCam_{}.csv'.format(prefix_reid)), index=False, header=True)
    if otherCam:
        outcomefilesOtherCam = find_files(path_main, '{}_outcome_otherCam_person_*_dataframe.csv'.format(prefix_reid), type='absolute')
        df_outcome_otherCam  = build_outcome_dataframe(outcomefilesOtherCam)
        print("df_outcome_otherCam shape: ",df_outcome_otherCam.shape)
        df_outcome_otherCam.to_csv(os.path.join(path_save,'skip_gather_df_outcomes_otherCam_{}.csv'.format(prefix_reid)), index=False, header=True)


# READ  and GATHER of Outcomes Re-id.

In [10]:
prefix_reid = 'SiamIDL'

In [11]:
gather_outcome_dataframes(path_main, path_save, prefix_reid, thisCam=True, otherCam=True)

df_outcome_thisCam shape:  (2011, 3)
df_outcome_otherCam shape:  (2011, 3)


In [12]:
prefix_reid = 'BoT'

In [13]:
gather_outcome_dataframes(path_main, path_save, prefix_reid, thisCam=True, otherCam=True)

df_outcome_thisCam shape:  (2011, 3)
df_outcome_otherCam shape:  (2011, 3)


# READ and GATHER of resultGUI

In [14]:
def gather_resultsGUI(path_main, path_save, prefix_reid, thisCam=False, otherCam=True):
    if thisCam:
        filesOtherCam = find_files(path_main, '{}_resultGUI_otherCam_person_*_seqVideo_*.csv'.format(prefix_reid), type='absolute')
        df_otherCam = build_dataframe(filesOtherCam)
        print(df_otherCam.shape)
        df_otherCam.to_csv(os.path.join(path_save,'skip_gather_df_resultGUI_otherCam_{}.csv'.format(prefix_reid)), index=False, header=True)

    if otherCam:
        filesThisCam  = find_files(path_main, '{}_resultGUI_thisCam_person_*_seqVideo_*.csv'.format(prefix_reid), type='absolute')
        df_thisCam = build_dataframe(filesThisCam)
        print(df_thisCam.shape)
        df_thisCam.to_csv(os.path.join(path_save,'skip_gather_df_resultGUI_thisCam_{}.csv'.format(prefix_reid)), index=False, header=True)


In [15]:
prefix_reid = 'SiamIDL'

In [16]:
gather_resultsGUI(path_main, path_save, prefix_reid, thisCam=True, otherCam=True)

(8760, 9)
(8760, 9)


In [17]:
prefix_reid = 'BoT'

In [18]:
gather_resultsGUI(path_main, path_save, prefix_reid, thisCam=True, otherCam=True)


(8760, 9)
(8760, 9)
